This project aims to detect graph isomomprhism from a given graph. We first generate a bunch of isomorphic graphs and non isomorphic graphs as our data. We train our neural network using these data, and then test the accuracy based on test data. 

In [291]:
import tensorflow as tf
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import isomorphism
import random as rnd
#generate an automorphism of a graph based in edges forms
def permute_graph(g):
    nodes = list(g.nodes())
    rnd.shuffle(nodes)
    edges = []
    for i,j in list(g.edges()):
        edges += [(nodes[i],nodes[j])]
    return edges

In [292]:
# function to return a matrix from a list of edges
def edges_to_matrix(edges, n):
    matrix = [[0 for i in range(n)] for j in range(n)]
    for i,j in edges:
        matrix[i][j] = 1
    return matrix

In [293]:
#Example of two isomoprhic graphs
n1,n2 = [0,1,2,3,4,5,6,7],[0,1,2,3,4,5,6,7]
e1 = [(0,2),(0,5),(0,4),(0,1),
      (1,0),(1,2),(1,5),(1,6),
      (2,0),(2,1),(2,5),(2,7),
      (3,5),(3,6),(4,0),(4,7),
      (5,0),(5,1),(5,2),(5,3),
      (6,3),(6,1),(7,2),(7,4)]
e2 = [(0,1),(0,7),(1,0),(1,2),
      (1,3),(1,6),(2,1),(2,3),
      (2,4),(2,6),(3,1),(3,2),
      (3,5),(3,6),(4,2),(4,5),
      (5,3),(5,4),(6,1),(6,2),
      (6,3),(6,7),(7,0),(7,6)]
g1,g2 = nx.Graph(),nx.Graph()
g1.add_nodes_from(n1);g1.add_edges_from(e1)
g2.add_nodes_from(n2);g1.add_edges_from(e2)

edges = permute_graph(g1)
a = nx.to_numpy_matrix(g1)
n = len(list(g1.nodes()))
edges_to_matrix(edges,n),a

([[0, 0, 1, 1, 0, 1, 1, 1],
  [1, 0, 1, 0, 1, 0, 1, 1],
  [0, 0, 0, 0, 0, 0, 1, 1],
  [0, 0, 0, 0, 0, 1, 1, 0],
  [1, 0, 0, 1, 0, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0]],
 matrix([[0., 1., 1., 0., 1., 1., 0., 1.],
         [1., 0., 1., 1., 0., 1., 1., 0.],
         [1., 1., 0., 1., 1., 1., 1., 1.],
         [0., 1., 1., 0., 0., 1., 1., 0.],
         [1., 0., 1., 0., 0., 1., 0., 1.],
         [1., 1., 1., 1., 1., 0., 0., 0.],
         [0., 1., 1., 1., 0., 0., 0., 1.],
         [1., 0., 1., 0., 1., 0., 1., 0.]]))

In [294]:
#generate isomorphic data first
size = 3000
n = 8
x = []
for i in range(size):
    edges = permute_graph(g1)
    x += [[np.array(edges_to_matrix(edges,n)),1]]
x

[[array([[0, 0, 0, 1, 0, 0, 1, 0],
         [0, 0, 0, 1, 1, 1, 0, 1],
         [1, 1, 0, 1, 0, 1, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 1],
         [1, 0, 0, 1, 1, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0]]), 1], [array([[0, 1, 1, 0, 0, 1, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 1, 1, 1, 1, 0],
         [0, 1, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0]]), 1], [array([[0, 0, 0, 0, 0, 1, 1, 0],
         [0, 0, 1, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [1, 1, 1, 0, 0, 1, 1, 0],
         [0, 1, 1, 1, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 1, 1, 1, 0]]), 1], [array([[0, 0, 0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
     

In [295]:
#generate non-isomorphic data. Simple idea: take one edge from g and call it h, and generate isomorphic graphs 
#to h. Surely, g and h are not isomorphic. 

edges_g = e1
size = 3000
n = 8
xx = []
for i in range(size):
    t = rnd.randint(1,len(edges_g)) # no of edges less
    rand_edges = set([rnd.randint(0,len(edges_g)) for i in range(t)]) # list of removed edges indices
    edges_h = [edges_g[i] for i in range(len(edges_g)) if i not in rand_edges]
    h = nx.Graph()
    h.add_nodes_from(n1)
    h.add_edges_from(edges_h) # extra shuffle to perumute the graph (isomorphic to non-isomorphic)
    edges = permute_graph(h)
    xx += [[np.array(edges_to_matrix(edges,n)),0]]
print(xx)

[[array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0]]), 0], [array([[0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]]), 0], [array([[0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]]), 0], [array([[0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 

In [296]:
#shuffle the data to mix isomorphic and non-isomorphic graphs
x_data = x + xx
rnd.shuffle(x_data)
for i in range(30):
    print(x_data[i])

[array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0]]), 0]
[array([[0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 1, 0]]), 0]
[array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 0, 1, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 1, 1],
       [1, 1, 0, 0, 0, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]]), 1]
[array([[0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0

In [297]:
size_train = int((size *4)/3)
print(size_train)
x_train,y_train = [],[]
for i in range(size_train):
    x_train += [x_data[i][0]]
    y_train += [x_data[i][1]]
     
x_train,y_train = np.array(x_train),np.array(y_train)
  
x_test,y_test = [],[]
for i in range(size_train,size*2):
    x_test += [x_data[i][0]]  
    y_test += [x_data[i][1]]
x_test,y_test = np.array(x_test),np.array(y_test)
x_test, y_test

4000


(array([[[0, 0, 0, ..., 0, 0, 1],
         [0, 0, 1, ..., 0, 0, 1],
         [1, 0, 0, ..., 0, 0, 1],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 1, 0, ..., 1, 1, 1],
         [0, 0, 0, ..., 0, 1, 0],
         [1, 1, 0, ..., 0, 1, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 1, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [1, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [1, 0, 0, ..., 1, 0, 0],
         [1, 1, 0, ..., 0, 1, 0]],
 
        ...,
 
        [[0, 1, 0, ..., 1, 1, 0],
         [0, 0, 0, ..., 0, 1, 0],
         [0, 0, 0, ..., 0, 1, 1],
         ...,
         [0, 1, 1, ..., 0, 1, 1],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 1, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],

In [298]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
4000/4000 [==============================] - 1s 334us/step - loss: 0.4024 - acc: 0.8598
Epoch 2/5
4000/4000 [==============================] - 1s 147us/step - loss: 0.0756 - acc: 1.0000
Epoch 3/5
4000/4000 [==============================] - 1s 142us/step - loss: 0.0208 - acc: 1.0000
Epoch 4/5
4000/4000 [==============================] - 1s 193us/step - loss: 0.0094 - acc: 1.0000
Epoch 5/5
2000/2000 [==============================] - 0s 224us/step


[0.004964238964021206, 1.0]

In [299]:
loss, accuracy = model.evaluate(x_test, y_test)

2000/2000 [==============================] - 0s 121us/step


In [300]:
x_test.shape

(2000, 8, 8)

In [301]:
gm = isomorphism.GraphMatcher(g1,g2)
print(gm.is_isomorphic(),"with isomorphism:", gm.mapping)

False with isomorphism: {}
